In [1]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern

## Assignment 4

## Task 1 

Implement probability of selection for single objective optimization. * (refer eqn.
12, 13, 14)

In [3]:
np.set_printoptions(precision=4,suppress=True) # easier to check values rn when we dont care too much about the exactness

In [81]:
# in minimization think probability of being wrong, in max probability of acceptance
# probability of sample A dominating sample B in max is P(A > B) is
def prob_selection(a, b, sigma_a, sigma_b):
    #sigma_b = sigma_b + 0.01 # just to prevent division by zero
    # seems not to be the correct way of hanlding sigma 0
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b
    
    top = -2.5*m / np.sqrt(2 + 2*s**2)
    bot = 1 + np.exp(top)
    return 1 / bot

# b clearly dominates
a = np.array([1.,1.,3.])
b = np.array([2.,5.,1.])
sigma_a = 0.01
sigma_b = 0.01

prob_selection(a, b, sigma_a, sigma_b)

array([0., 0., 1.])

In [82]:
def prob_sel2(a, b, sigma_a, sigma_b):
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b

    bot = 0.8*np.sqrt(2 + 2*s**2)
    return 0.5*(1 + np.tanh(m/bot))
    
    
    
# a1 clearly dominates b1, b1 clearly dominates a2
a = np.array([1.,1.,3.])
b = np.array([2.,5.,1.])
sigma_a = 1.
sigma_b = 1.

prob_sel2(a, b, sigma_a, sigma_b)

array([0.2227, 0.0067, 0.9241])

So we can see that probability of being wrong with first items (1 and 2) by choosing a is 0.22. Again for second items (1,5) thats 0.007.  And also with 3rd items (3,1) its 0.92. So we can clearly see its working properly.

## Task 2

Implement probability of dominance and ranking. * (refer eqn. 15, 17, 18, 22)

- Generate 50 samples from DTLZ2 problem (3 objectives, 5 decision variables) using LHS sampling
- Build Kriging surrogates using the generated samples
- Resample or generate 20 data points randomly
- Rank the 20 points using probability of dominance ranking

In [83]:
dtlz2_3 = test_problem_builder("DTLZ2", n_of_variables=5, n_of_objectives=3)

In [84]:
def create_samples(dec_dim, samples):
    # create samples 
    x = lhs(dec_dim, samples)     
    return np.asarray(x)

In [85]:
x = create_samples(5, 50)
print(x.shape)
eval_results = dtlz2_3.evaluate(x)
y = eval_results.objectives
print(y[:5])

(50, 5)
[[0.1371 0.0643 1.1041]
 [0.0247 0.9792 0.7709]
 [1.1628 0.121  0.2835]
 [1.2394 0.1599 0.6591]
 [0.9885 0.0187 0.5617]]


In [86]:
print(y[:,0])

[0.1371 0.0247 1.1628 1.2394 0.9885 0.0936 1.0715 0.181  0.0909 0.2334
 0.2436 0.1661 0.5621 0.3013 0.8812 0.6573 0.8101 0.0455 0.6783 0.9711
 0.136  0.9906 0.6532 0.8524 0.6053 0.5604 0.6017 0.7743 0.816  0.2302
 0.0322 0.1503 0.003  0.0155 0.8721 0.359  1.0353 1.4057 0.2536 0.3012
 0.3315 0.2029 0.5867 0.5506 0.3259 0.3336 0.0268 0.4476 0.4068 0.5986]


In [87]:
## build surrogates

kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 20.0), nu=1.5)
    
y1 = y[:,0]
gpr1 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y1)

y2 = y[:,1]
gpr2 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y2)

y3 = y[:,2]
gpr3 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y3)

In [88]:
## resampling

x_samples = create_samples(5, 20)

# predict
y1_sample, y1_std = gpr1.predict(x_samples, return_std=True) 
print(np.mean(y1_std))

# predict
y2_sample, y2_std = gpr2.predict(x_samples, return_std=True) 
print(np.mean(y2_std))
# predict
y3_sample, y3_std = gpr3.predict(x_samples, return_std=True) 
print(np.mean(y3_std))

y_samples = np.column_stack((y1_sample, y2_sample, y3_sample))
#y_samples.reshape(1,-1)
y_samples.shape

0.059758661192970276
0.06008813678032761
0.05121147283185466


(20, 3)

In [89]:
#print(y_samples)

In [90]:
def sum_of_rank_pos(n):
    return n*(n - 1)/2

sum_of_rank_pos(y_samples.shape[0])

190.0

In [91]:
# Papers example, just to test the code
A = [0.2, 0.9]  # tässä on a1 a2. kun lasketaan prod P (Ai < bi) i to n, tulee P(A < B)
B = [0.3, 0.5]
C = [0.5, 0.3]
D = [0.8, 0.2]
E = [0.85, 0.5]
F = [0.9, 0.95]

sols = np.array([A, B, C, D, E, F])
print(sols.shape)

(6, 2)


## DONE:

Single objective ranking works with uncertainty 0 and uncertainty 1. So prob_sel2 works
MO ranking with uncertainty 0 works

mo ranking with uncertainty works

## TODO:

rest of the assigment using some data

ask/find out about the bug with m.

In [92]:

## Need to:
## either multiply m by 10, or multiply sigmas by 10, to get similar results than in example
##


def prob_sel(a, b, sigma_a, sigma_b):
    s = sigma_a / sigma_b
    #print("S",s)
    #s = 1.
    m = (a - b) / sigma_b
    #print("M",m)
    m = m *10

    bot = 0.8*np.sqrt(2 + 2*s**2)
    res = 0.5*(1 + np.tanh(m/bot))
    res = np.prod(res)
    return res


# note 18.27 approx with uncertainty of 0 works
def sof_prob_dom_multi(fit, j, sigma_a, sigma_b):
    summa = 0
    # should just go over the multi
    for i in range(fit.shape[0]):
        for ii in range(fit.shape[1]):
            
            fits = fit[i][ii]
            if j[ii] == fits:
                #summa += 0.25
                s = prob_sel2(j[ii], fits, sigma_a, sigma_b)
                #print("eq", s)
                summa += 0.5*s
            elif j[ii] > fits:
                #summa += 0.5
                s = prob_sel2(j[ii], fits, sigma_a, sigma_b)
                #print("doms",s)
                summa += 0.5*s
            elif j[ii] < fits:
                #summa += 0
                s = prob_sel2(j[ii], fits, sigma_a, sigma_b)
                #print("does not dom",s)
                summa += 0.5*s
            else:
                print("should not get here")
        
    summa -= 0.5
    # this seems to need to be -5.5 for 5 var, 3 obj
    
    return summa

###  Testing with paper example

In [93]:
s2 = []
sigma_a, sigma_b = 0.001, 0.001 # no uncertainty

for i in range(len(sols)):
    s2.append(sof_prob_dom_multi(sols, sols[i], sigma_a, sigma_b))
s2

[2.0, 1.75, 1.5, 1.5, 3.25, 5.0]

In [94]:
np.sum(s2)

15.0

In [95]:
s3 = []
sigma_a, sigma_b = 1., 1. # uncertainty of 1. 

for i in range(len(sols)):
    s3.append(sof_prob_dom_multi(sols, sols[i], sigma_a, sigma_b))
s3

[2.4563903951090653,
 2.1842208625286634,
 2.1835448445715415,
 2.367724440504491,
 2.680534343436036,
 3.1275851138502024]

In [77]:
np.sum(s3)

15.0

## Ranking y_samples

In [96]:
s = []
sigma_a, sigma_b = np.mean(y1_std), np.mean(y2_std) # what about y3_std?


for i in range(len(y_samples)):
    s.append(sof_prob_dom_multi(y_samples, y_samples[i], sigma_a, sigma_b) - 5) # need to subtract -5 to hit 190 of sum rank pos, why?
s

[9.969902948052436,
 8.824946374506299,
 10.793185979834501,
 7.793596512366614,
 11.144231260213015,
 9.69023281210676,
 5.867692513312983,
 6.3545436690189,
 12.034783384098912,
 10.03338939676105,
 11.02243523077399,
 9.7834697053731,
 4.953107938432389,
 7.898109177434787,
 10.333824946205178,
 11.485039980555257,
 6.925057748638848,
 11.563144307016799,
 12.965306238121173,
 10.563999877177014]

In [97]:
np.sum(s)

190.0

In [98]:
sum_of_rank_pos(y_samples.shape[0])

190.0

### Everything kinda works?

* expect the m needs to be * by 10 for the paper examples to work

* and one should minus -5.5 from the sum with 5 var, 3 obj to keep sum of rank pos equal.


Probably has something to do with below..
"""
domination calculations.
If we consider the ranking case for a single dimension, if A and B are iden-
tical, we cannot choose between them and so add in 0.5 to the sum. With
non-domination, we also have the situation where we cannot choose between
objectives and should therefore add 0.5 to the sum as required. In the case of
uncertain measurements, we can multiply the value of 0:5 by the probability of
non-domination, and still subtract o 0:5 to allow for comparing the individual
with itself, thereby maintaining the sum of the rank positions as n(n 1)=2.
Thus we can add the non-domination term into (21). The rank calculation for
multi-objective ranking is shown in (22), where n is the number of measurements

""
Does it just come from that 5.5 happens to be the combinations of 3 dimensions in this case?